In [35]:
# pip install langchain langchain-openai pydantic
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

In [36]:
# 1) 구조화 출력 스키마 정의 (원하는 필드를 마음대로 바꿀 수 있어요)
class NewsSummary(BaseModel):
    title: str = Field(description="뉴스의 제목")
    date: str = Field(description="뉴스의 날짜")
    category: str = Field(description="뉴스 분야: 정치/경제/사회/스포츠/연예/기타")
    summary: str = Field(description="기사 전체 요약")
    key_points: list[str] = Field(description="핵심 문장 3~5개")


In [37]:
# 2) 파서와 포맷 지침 준비
parser = PydanticOutputParser(pydantic_object=NewsSummary)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"description": "뉴스의 제목", "title": "Title", "type": "string"}, "date": {"description": "뉴스의 날짜", "title": "Date", "type": "string"}, "category": {"description": "뉴스 분야: 정치/경제/사회/스포츠/연예/기타", "title": "Category", "type": "string"}, "summary": {"description": "기사 전체 요약", "title": "Summary", "type": "string"}, "key_points": {"description": "핵심 문장 3~5개", "items": {"type": "string"}, "title": "Key Points", "type": "array"}}, "required": ["title", "date", "category", "summary", "key_points"]}\n```'

In [38]:
# 3) 프롬프트 템플릿 (format 지침을 partial로 고정)
prompt = PromptTemplate(
    template=(
        "당신은 한국어 뉴스 요약 도우미입니다. 아래 기사를 읽고 "
        "요구하는 필드를 정확히 채워 구조화된 결과를 반환하세요.\n\n"
        "기사 원문:\n{article}\n\n"
        "출력 형식:\n{format_instructions}"
    ),
    input_variables=["article"],
    partial_variables={"format_instructions": format_instructions},
)

In [39]:
# 4) LLM 준비 (원하는 모델로 교체 가능)
llm = ChatOpenAI(temperature=0, model_name="gpt-4.1-mini")

In [40]:
# 5) 프롬프트 → 모델 → 파서 체인
chain = prompt | llm | parser

In [41]:
# 6) 기사 텍스트 (사용자 제공 내용)
article_text = """금어기 풀렸다…롯데마트, 살아있는 서해안 햇꽃게 판매 개시
부산 격포항·태안 신진도항 소재 어선과 사전 계약…역대 최대 물량 준비

유통입력 :2025/08/19 09:41    수정: 2025/08/19 10:20

김민아 기자 

롯데마트는 오는 21일 올해 첫 어획한 ‘서해안 햇꽃게’를 선보인다고 19일 밝혔다. 꽃게는 산란기 보호를 위해 매년 6월 21일부터 8월 20일까지 금어기를 시행하고 있어, 이번에 판매하는 상품은 금어기 해제 당일 새벽 서해안에서 잡아올린 물량이다.

오는 27일까지는 ‘서해안 햇꽃게(100g·냉장·국산)’를 행사 카드 결제시 20% 할인해 992원에 판매한다.

롯데마트는 새벽에 어획한 햇꽃게를 산지 직송해, 당일 오후 살아있는 상태로 공급한다. 조업 직후 꽃게를 5℃ 이하 냉수에 담가 기절시키고 모래톱과 유사한 환경을 조성해 전국 매장에 직송하는 구조다.

물량도 역대 최대로 준비했다. 롯데마트는 꽃게 주산지인 부안 격포항과 태안 신진도항 소재의 선단 40여 척과 사전 계약을 체결해, 전년 대비 약 30% 확대된 공급망을 구축했다.
"""

In [ ]:
# 7) 실행
result: NewsSummary = chain.invoke({"article": article_text})

In [43]:
# 8) 사용 예시
print(result)                  # Pydantic 객체로 바로 접근 가능
#print(result.model_dump())     # dict로 변환 (Pydantic v2)
#print(result.key_points)       # 리스트 필드 바로 활용

title='금어기 풀렸다…롯데마트, 살아있는 서해안 햇꽃게 판매 개시' date='2025-08-19' category='경제' summary='롯데마트는 8월 21일부터 올해 첫 어획한 서해안 햇꽃게를 판매한다. 금어기 해제 당일 새벽 잡은 꽃게를 당일 오후 살아있는 상태로 전국 매장에 직송하며, 역대 최대 물량을 준비했다. 부안 격포항과 태안 신진도항의 어선 40여 척과 사전 계약을 통해 전년 대비 약 30% 확대된 공급망을 구축했다. 8월 27일까지 행사 카드 결제 시 20% 할인된 가격에 판매한다.' key_points=['롯데마트는 오는 21일 올해 첫 어획한 서해안 햇꽃게를 선보인다.', '꽃게는 매년 6월 21일부터 8월 20일까지 금어기를 시행하며, 이번 판매는 금어기 해제 당일 새벽 잡은 물량이다.', '새벽에 어획한 꽃게를 5℃ 이하 냉수에 담가 기절시키고 모래톱과 유사한 환경을 조성해 당일 오후 살아있는 상태로 공급한다.', '부안 격포항과 태안 신진도항 소재 어선 40여 척과 사전 계약을 체결해 전년 대비 약 30% 확대된 공급망을 구축했다.', '8월 27일까지 행사 카드 결제 시 서해안 햇꽃게를 20% 할인된 992원에 판매한다.']


In [44]:
import json

print(json.dumps(result.model_dump(), indent=2, ensure_ascii=False))


{
  "title": "금어기 풀렸다…롯데마트, 살아있는 서해안 햇꽃게 판매 개시",
  "date": "2025-08-19",
  "category": "경제",
  "summary": "롯데마트는 8월 21일부터 올해 첫 어획한 서해안 햇꽃게를 판매한다. 금어기 해제 당일 새벽 잡은 꽃게를 당일 오후 살아있는 상태로 전국 매장에 직송하며, 역대 최대 물량을 준비했다. 부안 격포항과 태안 신진도항의 어선 40여 척과 사전 계약을 통해 전년 대비 약 30% 확대된 공급망을 구축했다. 8월 27일까지 행사 카드 결제 시 20% 할인된 가격에 판매한다.",
  "key_points": [
    "롯데마트는 오는 21일 올해 첫 어획한 서해안 햇꽃게를 선보인다.",
    "꽃게는 매년 6월 21일부터 8월 20일까지 금어기를 시행하며, 이번 판매는 금어기 해제 당일 새벽 잡은 물량이다.",
    "새벽에 어획한 꽃게를 5℃ 이하 냉수에 담가 기절시키고 모래톱과 유사한 환경을 조성해 당일 오후 살아있는 상태로 공급한다.",
    "부안 격포항과 태안 신진도항 소재 어선 40여 척과 사전 계약을 체결해 전년 대비 약 30% 확대된 공급망을 구축했다.",
    "8월 27일까지 행사 카드 결제 시 서해안 햇꽃게를 20% 할인된 992원에 판매한다."
  ]
}
